<a href="https://colab.research.google.com/github/hiza-daiki/TreeSpeciesClassification/blob/main/TreeSpeciesDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
PyTorchを使用して、提示されたファイル構成に基づき、樹皮画像を用いた樹種判別を行う画像認識の学習器を構築する。将来的なクラス増分（増分学習・知識蒸留）を視野に入れ、まずはStage 1のデータで学習・評価を行い、その後Stage 2のデータで増分学習を行い、最後に全てのクラスを含むデータで総合評価を行う。

## 環境構築

### Subtask:
PyTorch および必要なライブラリをインストールします。


**Reasoning**:
Install PyTorch and other necessary libraries using pip.



In [2]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install Pillow opencv-python pandas numpy

Looking in indexes: https://download.pytorch.org/whl/cu118


## データセットクラスとデータローダーの定義

### Subtask:
提示されたファイル構成に合わせて、各ステージのデータをロードするためのカスタムデータセットクラスとデータローダーを定義します。


**Reasoning**:
Define the custom dataset class and data loaders as per the instructions, including loading images, applying transformations, and handling labels.



In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class BarkDataset(Dataset):
    def __init__(self, base_dir, stage, split, transform=None):
        self.base_dir = base_dir
        self.stage = stage
        self.split = split # 'train', 'val', or 'test'
        self.transform = transform
        self.image_files = []
        self.labels = []
        self.classes = sorted(os.listdir(os.path.join(base_dir, f'stage_{stage}', split)))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

        for class_name in self.classes:
            class_dir = os.path.join(base_dir, f'stage_{stage}', split, class_name)
            for img_name in os.listdir(class_dir):
                self.image_files.append(os.path.join(class_dir, img_name))
                self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Assuming a base directory structure like:
# base_dir/
#   stage_1/
#     train/
#       class_a/
#       class_b/
#     val/
#       class_a/
#       class_b/
#     test/
#       class_a/
#       class_b/
#   stage_2/
#     train/
#       class_c/
#       class_d/
#     val/
#       class_c/
#       class_d/
#     test/
#       class_c/
#       class_d/
#   all_classes_test/ # For final evaluation
#     class_a/
#     class_b/
#     class_c/
#     class_d/


# Define data loaders for each stage and split
# Note: Replace 'path/to/your/data' with the actual base directory
base_data_dir = '/content/sample_data' # Placeholder, replace with actual path

# Stage 1
stage1_train_dataset = BarkDataset(base_data_dir, stage=1, split='train', transform=transform)
stage1_val_dataset = BarkDataset(base_data_dir, stage=1, split='val', transform=transform)
stage1_test_dataset = BarkDataset(base_data_dir, stage=1, split='test', transform=transform)

stage1_train_loader = DataLoader(stage1_train_dataset, batch_size=32, shuffle=True, num_workers=2)
stage1_val_loader = DataLoader(stage1_val_dataset, batch_size=32, shuffle=False, num_workers=2)
stage1_test_loader = DataLoader(stage1_test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Stage 2
stage2_train_dataset = BarkDataset(base_data_dir, stage=2, split='train', transform=transform)
stage2_val_dataset = BarkDataset(base_data_dir, stage=2, split='val', transform=transform)
stage2_test_dataset = BarkDataset(base_data_dir, stage=2, split='test', transform=transform)

stage2_train_loader = DataLoader(stage2_train_dataset, batch_size=32, shuffle=True, num_workers=2)
stage2_val_loader = DataLoader(stage2_val_dataset, batch_size=32, shuffle=False, num_workers=2)
stage2_test_loader = DataLoader(stage2_test_dataset, batch_size=32, shuffle=False, num_workers=2)

# All classes test data (assuming a separate directory for this)
# This part assumes a different directory structure for the final test set,
# where all classes are present in a single 'all_classes_test' directory.
# You might need to adjust the BarkDataset class or create a new one
# if your 'all_classes_test' structure is different.
# For simplicity, let's assume a structure similar to stage_1/test or stage_2/test
# but with all classes. A slight modification to BarkDataset might be needed
# to handle a directory structure like base_dir/all_classes_test/class_name/image.jpg

# To handle the 'all_classes_test' scenario, we can reuse BarkDataset but point
# it to the 'all_classes_test' directory and adjust how classes are discovered.
# We'll assume 'all_classes_test' is structured like stage directories.

class AllClassesTestDataset(Dataset):
    def __init__(self, base_dir, transform=None):
        self.base_dir = base_dir
        self.transform = transform
        self.image_files = []
        self.labels = []
        # Assuming all_classes_test has class subdirectories directly under it
        self.classes = sorted(os.listdir(os.path.join(base_dir, 'all_classes_test')))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

        for class_name in self.classes:
            class_dir = os.path.join(base_dir, 'all_classes_test', class_name)
            if os.path.isdir(class_dir): # Ensure it's a directory
                for img_name in os.listdir(class_dir):
                    self.image_files.append(os.path.join(class_dir, img_name))
                    self.labels.append(self.class_to_idx[class_name])


    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

all_classes_test_dataset = AllClassesTestDataset(base_data_dir, transform=transform)
all_classes_test_loader = DataLoader(all_classes_test_dataset, batch_size=32, shuffle=False, num_workers=2)

print("Custom Dataset and DataLoaders defined.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/stage_1/train'

## ベースモデルの定義

### Subtask:
樹種判別を行うためのベースとなるモデル（CNNなど）をPyTorchで定義します。


**Reasoning**:
Define a simple Convolutional Neural Network (CNN) model using `torch.nn.Module` for image classification, considering the input image size and the number of classes from Stage 1.



In [ ]:
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Assuming input image size is 224x224 after transformations
        # Calculate the size of the flattened features after pooling
        # 224 / 2 / 2 / 2 = 28
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))

        x = x.view(x.size(0), -1) # Flatten the output for the fully connected layers
        x = self.dropout(self.relu4(self.fc1(x)))
        x = self.fc2(x)
        return x

# Determine the number of classes for Stage 1.
# This requires the stage1_train_dataset to be successfully instantiated,
# which failed in the previous step due to missing data.
# We will use a placeholder value for now, assuming Stage 1 has 2 classes based on the previous step's assumption of 'class_a' and 'class_b'.
# In a real scenario, this would be determined from the dataset object: stage1_train_dataset.num_classes

num_classes_stage1 = 2 # Placeholder - replace with actual number of classes in Stage 1

# Create an instance of the model
model = SimpleCNN(num_classes=num_classes_stage1)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Model defined and moved to {device}.")
print(model)

Model defined and moved to cpu.
SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (relu4): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


## ステージ1での学習

### Subtask:
Stage 1のデータ (`stage_1/train`) を使用してベースモデルを学習させます。


**Reasoning**:
Define the loss function and optimizer, set up the training and validation loop, and train the model on the stage 1 data.



In [ ]:
import torch.optim as optim
import torch.nn as nn
import time

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set up training parameters
num_epochs = 10
best_val_accuracy = 0.0

print("Starting training...")

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training loop
    for i, data in enumerate(stage1_train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(stage1_train_loader)
    epoch_accuracy = 100 * correct_train / total_train
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_accuracy:.2f}%')

    # Validation step
    model.eval()  # Set model to evaluation mode
    correct_val = 0
    total_val = 0
    val_loss = 0.0

    with torch.no_grad():
        for data in stage1_val_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(stage1_val_loader)
    val_accuracy = 100 * correct_val / total_val
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    # Save the model if validation accuracy improves
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_stage1_model.pth')
        print("Saved best model state dictionary.")

print("Finished Training.")

Starting training...


NameError: name 'stage1_train_loader' is not defined

**Reasoning**:
The error indicates that `stage1_train_loader` is not defined. This is because the previous step where the data loaders were defined failed due to missing data files. Although the definition itself was conceptually correct, the instantiation failed. To proceed with training, we need to re-execute the data loader definition step. Since the goal is to train the model, and the model definition was successful, we will redefine the data loaders and then proceed with the training loop in a single code block. We will include the necessary imports from the previous successful steps.



In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import time

# Redefine the BarkDataset class and DataLoaders (as the previous attempt failed)
class BarkDataset(Dataset):
    def __init__(self, base_dir, stage, split, transform=None):
        self.base_dir = base_dir
        self.stage = stage
        self.split = split # 'train', 'val', or 'test'
        self.transform = transform
        self.image_files = []
        self.labels = []
        # Adjusted to handle potential missing directories more gracefully, though it will still error if base_dir is wrong
        stage_split_dir = os.path.join(base_dir, f'stage_{stage}', split)
        if not os.path.exists(stage_split_dir):
             raise FileNotFoundError(f"Directory not found: {stage_split_dir}")

        self.classes = sorted([d for d in os.listdir(stage_split_dir) if os.path.isdir(os.path.join(stage_split_dir, d))])
        if not self.classes:
             raise ValueError(f"No class directories found in {stage_split_dir}")

        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

        for class_name in self.classes:
            class_dir = os.path.join(stage_split_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.image_files.append(os.path.join(class_dir, img_name))
                self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return None, None # Handle potential image loading errors
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Assuming a base directory structure like:
# base_dir/
#   stage_1/
#     train/
#       class_a/
#       class_b/
#     val/
#       class_a/
#       class_b/
#     test/
#       class_a/
#       class_b/
#   stage_2/
#     train/
#       class_c/
#       class_d/
#     val/
#       class_c/
#       class_d/
#     test/
#       class_c/
#       class_d/
#   all_classes_test/ # For final evaluation
#     class_a/
#     class_b/
#     class_c/
#     class_d/


# Define data loaders for each stage and split
# Note: Replace 'path/to/your/data' with the actual base directory
# Using a placeholder again, as the actual data path is unknown. This will likely fail again
# if the data is not present, but it's necessary to attempt to define the loaders
# before the training loop.
base_data_dir = '/content/sample_data' # Placeholder, replace with actual path

try:
    # Stage 1
    stage1_train_dataset = BarkDataset(base_data_dir, stage=1, split='train', transform=transform)
    stage1_val_dataset = BarkDataset(base_data_dir, stage=1, split='val', transform=transform)
    stage1_test_dataset = BarkDataset(base_data_dir, stage=1, split='test', transform=transform)

    stage1_train_loader = DataLoader(stage1_train_dataset, batch_size=32, shuffle=True, num_workers=2)
    stage1_val_loader = DataLoader(stage1_val_dataset, batch_size=32, shuffle=False, num_workers=2)
    stage1_test_loader = DataLoader(stage1_test_dataset, batch_size=32, shuffle=False, num_workers=2)

    # Stage 2 (defined for completeness, though not used in this subtask)
    stage2_train_dataset = BarkDataset(base_data_dir, stage=2, split='train', transform=transform)
    stage2_val_dataset = BarkDataset(base_data_dir, stage=2, split='val', transform=transform)
    stage2_test_dataset = BarkDataset(base_data_dir, stage=2, split='test', transform=transform)

    stage2_train_loader = DataLoader(stage2_train_dataset, batch_size=32, shuffle=True, num_workers=2)
    stage2_val_loader = DataLoader(stage2_val_dataset, batch_size=32, shuffle=False, num_workers=2)
    stage2_test_loader = DataLoader(stage2_test_dataset, batch_size=32, shuffle=False, num_workers=2)

    # All classes test data (defined for completeness, though not used in this subtask)
    class AllClassesTestDataset(Dataset):
        def __init__(self, base_dir, transform=None):
            self.base_dir = base_dir
            self.transform = transform
            self.image_files = []
            self.labels = []
            all_classes_dir = os.path.join(base_dir, 'all_classes_test')
            if not os.path.exists(all_classes_dir):
                 raise FileNotFoundError(f"Directory not found: {all_classes_dir}")

            self.classes = sorted([d for d in os.listdir(all_classes_dir) if os.path.isdir(os.path.join(all_classes_dir, d))])
            if not self.classes:
                raise ValueError(f"No class directories found in {all_classes_dir}")

            self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

            for class_name in self.classes:
                class_dir = os.path.join(all_classes_dir, class_name)
                if os.path.isdir(class_dir): # Ensure it's a directory
                    for img_name in os.listdir(class_dir):
                        self.image_files.append(os.path.join(class_dir, img_name))
                        self.labels.append(self.class_to_idx[class_name])

        def __len__(self):
            return len(self.image_files)

        def __getitem__(self, idx):
            img_path = self.image_files[idx]
            try:
                image = Image.open(img_path).convert('RGB')
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
                return None, None # Handle potential image loading errors

            label = self.labels[idx]

            if self.transform:
                image = self.transform(image)

            return image, label

    all_classes_test_dataset = AllClassesTestDataset(base_data_dir, transform=transform)
    all_classes_test_loader = DataLoader(all_classes_test_dataset, batch_size=32, shuffle=False, num_workers=2)

    print("Custom Dataset and DataLoaders defined.")

    # Define loss function and optimizer (re-defined for clarity in this block)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Set up training parameters (re-defined for clarity)
    num_epochs = 10
    best_val_accuracy = 0.0

    print("Starting training...")

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        # Training loop
        for i, data in enumerate(stage1_train_loader):
            inputs, labels = data
            # Filter out None values in case of image loading errors
            if inputs is None or labels is None:
                continue
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(stage1_train_loader)
        epoch_accuracy = 100 * correct_train / total_train
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_accuracy:.2f}%')

        # Validation step
        model.eval()  # Set model to evaluation mode
        correct_val = 0
        total_val = 0
        val_loss = 0.0

        with torch.no_grad():
            for data in stage1_val_loader:
                images, labels = data
                # Filter out None values
                if images is None or labels is None:
                    continue
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(stage1_val_loader)
        val_accuracy = 100 * correct_val / total_val
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

        # Save the model if validation accuracy improves
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_stage1_model.pth')
            print("Saved best model state dictionary.")

    print("Finished Training.")

except FileNotFoundError as e:
    print(f"Error: {e}. Please ensure the data directory structure is correct.")
    # The task cannot be completed without the data loaders being successfully instantiated.
    # We'll need to signal failure or request the user to provide the correct path/data.
    # As per instructions, we should not ask for user input.
    # We will finish the task with failure because the prerequisite (data loading) failed.

except ValueError as e:
    print(f"Error: {e}. Please ensure there are class subdirectories in your data folders.")
    # Similar to FileNotFoundError, cannot proceed without valid data.

except Exception as e:
    print(f"An unexpected error occurred: {e}")
    # Catch any other unexpected errors during data loading or training.


Error: Directory not found: /content/sample_data/stage_1/train. Please ensure the data directory structure is correct.


データセットクラス `BarkDataset` および `AllClassesTestDataset` の定義に基づくと、データは以下のファイル構成になっている必要があります。

## ステージ1での評価

### Subtask:
学習済みモデルをStage 1のテストデータ (`stage_1/test`) で評価し、初期性能を確認します。


**Reasoning**:
Evaluate the trained model on the Stage 1 test data by setting the model to evaluation mode, iterating through the test loader, making predictions, and calculating accuracy and a confusion matrix. This requires the model and the test loader to be available from previous steps. Since the training step failed due to data loading issues, this step will also likely fail unless the data path is corrected.



In [ ]:
import torch
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

# Check if model and stage1_test_loader exist from previous steps
# If the previous training step failed due to data loading, these might not be defined.
# We will assume they exist based on the flow, but the code will fail if they don't.

if 'model' not in locals() or 'stage1_test_loader' not in locals():
    print("Error: Model or stage1_test_loader not found. Ensure previous steps (defining model and data loaders) were successful.")
else:
    # Set model to evaluation mode
    model.eval()

    all_labels = []
    all_predictions = []

    print("Starting evaluation on Stage 1 test data...")

    # Evaluate the model on the test data
    with torch.no_grad():
        for data in stage1_test_loader:
            images, labels = data
            # Filter out None values in case of image loading errors
            if images is None or labels is None:
                continue
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    # Calculate evaluation metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    conf_matrix = confusion_matrix(all_labels, all_predictions)

    print(f'Stage 1 Test Accuracy: {accuracy:.4f}')
    print('Stage 1 Confusion Matrix:')
    print(conf_matrix)

    # Determine class names if possible (assuming stage1_test_dataset was created successfully)
    if 'stage1_test_dataset' in locals():
        print('Stage 1 Class Names:')
        print(stage1_test_dataset.classes)

print("Finished evaluation.")

Error: Model or stage1_test_loader not found. Ensure previous steps (defining model and data loaders) were successful.
Finished evaluation.


## 増分学習の準備

### Subtask:
Stage 2のデータ (`stage_2/train`) を使用して増分学習を行うための準備をします。必要に応じて、知識蒸留のための設定も検討します。


**Reasoning**:
The previous steps failed because the data loaders could not be instantiated due to a `FileNotFoundError` and `ValueError` related to the data directory structure. Before proceeding with Stage 2 incremental learning, it is crucial to ensure that the Stage 2 data loader (`stage2_train_loader`) can be successfully created. This involves verifying and potentially correcting the `base_data_dir` and the `BarkDataset` definition's ability to locate Stage 2 training data. I will reuse the `BarkDataset` and `DataLoader` definitions from the previous attempts and attempt to instantiate the `stage2_train_dataset` and `stage2_train_loader` again. I will also check if the `model` object exists from the previous steps, as it is needed for incremental learning. If the data loading is successful, I will set up the teacher model for knowledge distillation and define a new optimizer and loss function as requested by the instructions.



In [ ]:
import torch.optim as optim
import torch.nn.functional as F # For KL Divergence

# Re-verify the base_data_dir and attempt to instantiate Stage 2 data loader
# The BarkDataset and DataLoader classes are assumed to be defined in a previous cell.
# If that cell failed, this will fail again.

# Using the placeholder base_data_dir again. This needs to be correct for the code to run.
base_data_dir = '/content/sample_data' # Placeholder, replace with actual path

try:
    # Attempt to instantiate Stage 2 train dataset and loader
    stage2_train_dataset = BarkDataset(base_data_dir, stage=2, split='train', transform=transform)
    stage2_train_loader = DataLoader(stage2_train_dataset, batch_size=32, shuffle=True, num_workers=2)
    print("Stage 2 train dataset and loader successfully created.")

    # Check if the model exists from previous steps
    if 'model' in locals():
        print("Model instance found.")
        # 3. Set Stage 1 model as teacher model and switch to evaluation mode
        teacher_model = model
        teacher_model.eval()
        print("Stage 1 model set as teacher model and set to evaluation mode.")

        # 4. Define a new optimizer for incremental learning (Stage 2)
        # Using Adam with a potentially different learning rate.
        incremental_optimizer = optim.Adam(model.parameters(), lr=0.0005)
        print("New optimizer for incremental learning defined.")

        # 5. Define loss function(s)
        # Cross-entropy loss for the student model
        criterion = nn.CrossEntropyLoss()
        print("Cross-entropy loss defined.")

        # If knowledge distillation is used, define KL divergence loss
        # Assuming knowledge distillation will be implemented in the training loop.
        # KL divergence loss will be used to match student output probabilities to teacher output probabilities.
        # criterion_kd = nn.KLDivLoss(reduction='batchmean') # Example KL divergence loss
        # print("KL Divergence loss for knowledge distillation considered.")

    else:
        print("Error: Model instance not found. Ensure the model definition step was successful.")
        # Cannot proceed without the model.
        # This will lead to a failure in completing the subtask.

except FileNotFoundError as e:
    print(f"Error: {e}. Please ensure the data directory structure for Stage 2 train data is correct.")
    # Cannot proceed without the data loaders being successfully instantiated.
    # This will lead to a failure in completing the subtask.

except ValueError as e:
    print(f"Error: {e}. Please ensure there are class subdirectories in your Stage 2 train data folder.")
    # Cannot proceed without valid data.
    # This will lead to a failure in completing the subtask.

except Exception as e:
    print(f"An unexpected error occurred: {e}")
    # Catch any other unexpected errors.
    # This will lead to a failure in completing the subtask.


Error: Directory not found: /content/sample_data/stage_2/train. Please ensure the data directory structure for Stage 2 train data is correct.


## 増分学習

### Subtask:
Stage 1で学習したモデルをベースに、Stage 2のデータを使用して増分学習を行います。知識蒸留を導入する場合は、この段階で適用します。


**Reasoning**:
Based on the instruction to perform incremental learning using Stage 2 data and the history of failures due to data loading issues, the first crucial step is to attempt to load the Stage 2 training data again. I will re-use the existing `BarkDataset` and `DataLoader` definitions (assuming they are available from previous successful code cells) and attempt to instantiate the `stage2_train_loader`. If successful, I will then proceed to load the previously saved Stage 1 model, prepare it for incremental learning, define the optimizer and loss function, and execute the incremental training loop as outlined in the instructions. Knowledge distillation will be incorporated if a teacher model is available.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import os
import torch.nn.functional as F # For KL Divergence if using KD

# Re-using the dataset and dataloader definitions from previous steps.
# If the previous definition cell failed, this will fail again.
# Assuming base_data_dir is defined, but confirming its existence is crucial.

# Let's explicitly check if the necessary data loader definition was successful previously
# by attempting to access the variable name. If it doesn't exist, we'll know the data loading failed.
try:
    # Attempt to access stage2_train_loader to see if it was successfully created before
    _ = stage2_train_loader
    print("stage2_train_loader found from previous steps.")

except NameError:
    print("stage2_train_loader not found. Attempting to re-create DataLoaders.")
    # If the loader was not found, it means the previous data loading steps failed.
    # We need to re-attempt defining the dataset and dataloader here.
    # This assumes the BarkDataset and transformation definitions are available globally from previous successful cells.

    # Using the placeholder base_data_dir again. This needs to be correct for the code to run.
    base_data_dir = '/content/sample_data' # Placeholder, replace with actual path
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    try:
        stage2_train_dataset = BarkDataset(base_data_dir, stage=2, split='train', transform=transform)
        stage2_train_loader = DataLoader(stage2_train_dataset, batch_size=32, shuffle=True, num_workers=2)
        print("Stage 2 train dataset and loader successfully created.")
    except FileNotFoundError as e:
        print(f"Error re-creating Stage 2 data loader: {e}. Please ensure the data directory structure for Stage 2 train data is correct.")
        # Cannot proceed without the data loaders being successfully instantiated.
        # Signal a critical error and potentially exit or finish with failure later.
        stage2_train_loader = None # Ensure the variable is None if creation fails
    except ValueError as e:
        print(f"Error re-creating Stage 2 data loader: {e}. Please ensure there are class subdirectories in your Stage 2 train data folder.")
        stage2_train_loader = None # Ensure the variable is None if creation fails
    except Exception as e:
        print(f"An unexpected error occurred during Stage 2 data loader re-creation: {e}")
        stage2_train_loader = None # Ensure the variable is None if creation fails


# Proceed with incremental learning only if the data loader is available
if stage2_train_loader is not None:
    # Check if the model exists from previous steps
    if 'model' in locals():
        print("Model instance found.")
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device) # Ensure model is on the correct device

        # Load the best Stage 1 model weights if saved
        stage1_model_path = 'best_stage1_model.pth'
        if os.path.exists(stage1_model_path):
            print(f"Loading Stage 1 model weights from {stage1_model_path}")
            try:
                model.load_state_dict(torch.load(stage1_model_path, map_location=device))
                print("Stage 1 model weights loaded successfully.")
            except Exception as e:
                print(f"Error loading Stage 1 model weights: {e}")
                # Continue with the potentially untrained model or handle as failure
        else:
            print(f"Warning: {stage1_model_path} not found. Starting incremental learning with potentially untrained or partially trained model.")


        # 3. Set Stage 1 model as teacher model if knowledge distillation is used
        # Create a separate instance for the teacher if we need to preserve its state.
        # For simplicity, we'll just use the 'model' instance and ensure it's in eval mode when used as teacher.
        teacher_model = model # Using the same model instance as teacher
        teacher_model.eval() # Set teacher model to evaluation mode
        print("Stage 1 model set as teacher model (in eval mode).")

        # Determine the number of classes for Stage 2.
        # This requires the stage2_train_dataset to be successfully instantiated.
        # If Stage 2 has new classes, the model's final layer needs modification.
        num_classes_stage2 = len(stage2_train_dataset.classes)
        print(f"Number of classes in Stage 2 train data: {num_classes_stage2}")

        # Important: If Stage 2 introduces new classes, the output layer of the model needs to be adapted.
        # This is a crucial part of incremental learning for class-incremental scenarios.
        # Assuming for now that Stage 2 either adds new classes or uses existing ones.
        # If new classes are added, the final layer needs to be reinitialized or expanded.

        # Simple approach: Replace the final layer if the number of classes changes
        # This might require re-training the new layer from scratch or with a low learning rate.
        # A more sophisticated approach might involve adding a new head or using techniques like freezing layers.
        if model.fc2.out_features != num_classes_stage2:
             print(f"Adapting model output layer from {model.fc2.out_features} to {num_classes_stage2} classes.")
             # Create a new fully connected layer
             new_fc = nn.Linear(model.fc2.in_features, num_classes_stage2).to(device)
             # Replace the old final layer
             model.fc2 = new_fc
             print("Model output layer adapted.")
             # Note: Parameters of the new layer are randomly initialized by default.
             # You might want to initialize them differently or train them specifically.

        # 4. Define a new optimizer for incremental learning (Stage 2)
        # Using Adam with a potentially different learning rate, optimizing all model parameters.
        # If specific layers were frozen or modified, you might want to optimize only the modified/unfrozen ones.
        incremental_optimizer = optim.Adam(model.parameters(), lr=0.0005)
        print("Optimizer for incremental learning defined.")

        # 5. Define loss function(s)
        # Cross-entropy loss for the student model
        criterion_ce = nn.CrossEntropyLoss()
        print("Cross-entropy loss defined.")

        # Define KL divergence loss for knowledge distillation
        # Temperature parameter for softening the teacher's output
        temperature = 2.0 # Example temperature value
        criterion_kd = nn.KLDivLoss(reduction='batchmean')
        print(f"KL Divergence loss for knowledge distillation defined with temperature {temperature}.")

        # Define alpha weight for balancing CE and KD loss
        alpha = 0.5 # Example weight for KD loss (0.0 means only CE, 1.0 means only KD)
        print(f"Alpha weight for knowledge distillation set to {alpha}.")


        # 6. Set up training parameters for incremental learning
        num_epochs_incremental = 5 # Number of epochs for incremental training
        print(f"Starting incremental training for {num_epochs_incremental} epochs...")

        # Incremental Training loop
        for epoch in range(num_epochs_incremental):
            model.train()  # Set model to training mode
            running_loss_incremental = 0.0
            correct_train_incremental = 0
            total_train_incremental = 0

            start_time = time.time()

            for i, data in enumerate(stage2_train_loader):
                inputs, labels = data
                # Filter out None values in case of image loading errors
                if inputs is None or labels is None:
                    continue
                inputs, labels = inputs.to(device), labels.to(device)

                incremental_optimizer.zero_grad()

                # Forward pass - get student output
                student_outputs = model(inputs)

                # Calculate Cross-Entropy Loss
                ce_loss = criterion_ce(student_outputs, labels)

                # Calculate Knowledge Distillation Loss
                # Get teacher output (no gradients needed for teacher)
                with torch.no_grad():
                    teacher_outputs = teacher_model(inputs)

                # Apply temperature and softmax to both student and teacher outputs
                student_outputs_soft = F.log_softmax(student_outputs / temperature, dim=1)
                teacher_outputs_soft = F.softmax(teacher_outputs / temperature, dim=1)

                # Calculate KL Divergence loss
                kd_loss = criterion_kd(student_outputs_soft, teacher_outputs_soft)

                # Combine losses
                # Total loss = (1 - alpha) * CE_loss + alpha * KD_loss * temperature^2
                # The temperature^2 scaling is often applied to the KD loss when using KLDivLoss
                # with log_softmax on the student output and softmax on the teacher output.
                loss = (1 - alpha) * ce_loss + alpha * kd_loss * (temperature ** 2)


                # Backward pass and optimize
                loss.backward()
                incremental_optimizer.step()

                running_loss_incremental += loss.item()

                _, predicted = torch.max(student_outputs.data, 1)
                total_train_incremental += labels.size(0)
                correct_train_incremental += (predicted == labels).sum().item()

            epoch_loss_incremental = running_loss_incremental / len(stage2_train_loader)
            epoch_accuracy_incremental = 100 * correct_train_incremental / total_train_incremental
            epoch_time = time.time() - start_time

            print(f'Incremental Epoch [{epoch+1}/{num_epochs_incremental}], Loss: {epoch_loss_incremental:.4f}, Accuracy: {epoch_accuracy_incremental:.2f}%, Time: {epoch_time:.2f}s')

            # Optional: Add validation step here using stage2_val_loader if available
            # and save the model if performance improves.
            # This would require checking if stage2_val_loader was successfully created.


        print("Finished Incremental Training.")

        # Save the incrementally trained model
        torch.save(model.state_dict(), 'incrementally_trained_model_stage2.pth')
        print("Saved incrementally trained model state dictionary.")

    else:
        print("Cannot perform incremental training: Model instance not found.")
        # If model is not found, the previous model definition step failed.

else:
    print("Cannot perform incremental training: Stage 2 train data loader not available.")


stage2_train_loader not found. Attempting to re-create DataLoaders.
Error re-creating Stage 2 data loader: Directory not found: /content/sample_data/stage_2/train. Please ensure the data directory structure for Stage 2 train data is correct.
Cannot perform incremental training: Stage 2 train data loader not available.


## 総合評価

### Subtask:
増分学習後のモデルを全てのクラスを含むテストデータ (`test_all`) で評価し、モデル全体の性能を確認します。


**Reasoning**:
Evaluate the incrementally trained model on the test data containing all classes. This involves setting the model to evaluation mode, iterating through the all-classes test loader, making predictions, and calculating evaluation metrics like accuracy and confusion matrix. The code will check if the model and the all-classes test loader exist from previous steps and handle potential file loading errors.



In [ ]:
import torch
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import os

# Check if model and all_classes_test_loader exist from previous steps.
# If previous steps failed due to data loading, these might not be defined or the model might be untrained.
# We need to gracefully handle these cases.

if 'model' not in locals():
    print("Error: Model instance not found. Ensure the model definition step was successful.")
    # If the model doesn't exist, we cannot proceed with evaluation.
    evaluation_successful = False
elif 'all_classes_test_loader' not in locals() or all_classes_test_loader is None:
     print("Error: all_classes_test_loader not found or not successfully created. Ensure data loading for all classes test data was successful.")
     # If the data loader doesn't exist or failed creation, we cannot proceed.
     evaluation_successful = False
else:
    evaluation_successful = True
    print("Model instance and all_classes_test_loader found.")

    # Proceed with evaluation only if prerequisites are met
    if evaluation_successful:
        # Set model to evaluation mode
        model.eval()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device) # Ensure model is on the correct device

        # Load the incrementally trained model weights if saved
        incremental_model_path = 'incrementally_trained_model_stage2.pth'
        if os.path.exists(incremental_model_path):
            print(f"Loading incrementally trained model weights from {incremental_model_path}")
            try:
                model.load_state_dict(torch.load(incremental_model_path, map_location=device))
                print("Incrementally trained model weights loaded successfully.")
            except Exception as e:
                print(f"Error loading incrementally trained model weights: {e}")
                print("Proceeding with the current state of the model, which might be Stage 1 trained or untrained.")
        else:
            print(f"Warning: {incremental_model_path} not found. Evaluating the current state of the model (likely Stage 1 trained or untrained).")


        all_labels = []
        all_predictions = []

        print("Starting evaluation on all classes test data...")

        # Evaluate the model on the test data
        with torch.no_grad():
            for data in all_classes_test_loader:
                images, labels = data
                # Filter out None values in case of image loading errors
                if images is None or labels is None:
                    continue
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())

        # Calculate evaluation metrics
        if len(all_labels) > 0:
            accuracy = accuracy_score(all_labels, all_predictions)
            conf_matrix = confusion_matrix(all_labels, all_predictions)

            print(f'Overall Test Accuracy (All Classes): {accuracy:.4f}')
            print('Overall Confusion Matrix (All Classes):')
            print(conf_matrix)

            # Determine class names if possible (assuming all_classes_test_dataset was created successfully)
            if 'all_classes_test_dataset' in locals() and all_classes_test_dataset is not None:
                print('All Classes:')
                print(all_classes_test_dataset.classes)
            else:
                print("Could not retrieve class names from the dataset.")

        else:
            print("No data available for evaluation. Ensure all_classes_test_loader is populated.")


print("Finished evaluation attempt.")


Error: all_classes_test_loader not found or not successfully created. Ensure data loading for all classes test data was successful.
Finished evaluation attempt.


## 結果の分析と考察

### Subtask:
学習過程、評価結果を分析し、増分学習や知識蒸留の効果、モデルの課題などを考察します。


## Summary:

### Data Analysis Key Findings

*   The entire data analysis process failed at multiple steps due to a `FileNotFoundError`.
*   Specifically, attempts to instantiate `BarkDataset` and `AllClassesTestDataset` for Stage 1, Stage 2, and all-classes test data failed because the base directory `/content/sample_data` did not contain the expected subdirectories (`stage_1`, `stage_2`, `all_classes_test`) with the required data structure.
*   As a consequence of the data loading failures, the subsequent steps for model training (Stage 1 and incremental Stage 2) and evaluation (Stage 1 test and all-classes test) could not be executed.
*   While the code for defining the model, optimizer, loss functions (including KL Divergence for potential knowledge distillation), and the training/evaluation loops was written, its functionality could not be verified due to the lack of data.

### Insights or Next Steps

*   **Critical Next Step:** The absolute necessary next step is to ensure the data is correctly placed at the specified `base_data_dir` (`/content/sample_data` or an updated path) following the expected directory structure (`stage_1/[train|val|test]/class_name/`, `stage_2/[train|val|test]/class_name/`, `all_classes_test/class_name/`).
*   Once the data is correctly set up, re-run the entire process sequentially to confirm successful data loading, model training on Stage 1, evaluation on Stage 1, incremental training on Stage 2 (with or without knowledge distillation), and finally, comprehensive evaluation on the all-classes test set.
